## Figure 1: plot_full_cartoon of LABLDH


In [1]:
import os, LocalEnv, stats_utils
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

stats_utils.py:111: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'agg' by the following code:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/ipykernel/kernelapp.py"

In [2]:
curr_version = '10000-episodes-lastnormal'
inverse01 = True # Setting 'True' to interpret 'Normal' as 'Negative'

inverse_maker = '_inversed01' if inverse01 else ''

In [3]:
lab = 'LABLDH'
data_source = 'Stanford'
lab_type = 'panel'
score_thres = 0.756
include_threshold_colors = True

project_folderpath = os.path.join(LocalEnv.PATH_TO_CDSS, 'scripts/LabTestAnalysis')
stats_folderpath = os.path.join(project_folderpath, 'lab_statistics')
ML_folderpath = os.path.join(project_folderpath, 'machine_learning')


ML_folderpath = os.path.join(project_folderpath, 'machine_learning')

In [4]:

dataSet_foldername = 'data-%s-%s-%s'%(data_source, 
                                      lab_type, 
                                      curr_version)
dataML_folderpath = os.path.join(ML_folderpath, dataSet_foldername)
dataStats_folderpath = os.path.join(stats_folderpath, dataSet_foldername)

all_labs = stats_utils.get_all_labs(data_source, lab_type)

labDescriptions = stats_utils.get_lab_descriptions(data_source=data_source, 
                                                   lab_type=lab_type)

In [5]:
figure2_folderpath = os.path.join(stats_folderpath, 'Fig2_Order_Intensities')

## Order_Intensities

In [6]:
import stats_database
labs_cnts_order_1day = stats_database.TOP_PANELS_AND_COUNTS_IN_1DAY[:20]
labs_order_1day = [x[0] for x in labs_cnts_order_1day]

labs_guideline = ['LABESRP', 'LABALB', 'LABA1C', 'LABTSH', 'LABCRP', 'LABYCP', 'LABLIPS', 'LABAGALA', 'LABNTBNP']


use_cached_fig_data = True
DEFAULT_TIMELIMIT = stats_utils.DEFAULT_TIMELIMIT



scale = 1. / stats_utils.NUM_DISTINCT_ENCS * 1000

lab_type = 'panel'

to_annotate_percentages=False

cached_result_foldername = os.path.join(stats_folderpath, 'Fig2_Order_Intensities/')
if not os.path.exists(cached_result_foldername):
    os.mkdir(cached_result_foldername)

In [7]:
params_1 = {'labs':labs_order_1day,
            'result_label':'labs_order_1day',
            'scale_method' : 'by_scale',
            'include_legend':True
           }

params_2 = {'labs':labs_guideline,
            'result_label':'labs_guideline',
            'scale_method' : 'normalize',
            'include_legend':False
           }

params = params_1

labs = params['labs'] #  # 
print "Labs to be plot:", labs
result_label= params['result_label'] # 'labs_guideline'# 


scale_method=params['scale_method']
include_legend=params['include_legend']

lab_descriptions = stats_utils.get_lab_descriptions(data_source=data_source, lab_type=lab_type)



cached_result_filename = 'Order_Intensities_%s_%s.csv'%(lab_type, result_label)
cached_result_path = os.path.join(cached_result_foldername, cached_result_filename)


Labs to be plot: ['LABMETB', 'LABMGN', 'LABCBCD', 'LABLACWB', 'LABPTT', 'LABPHOS', 'LABPT', 'LABCAI', 'LABK', 'LABHEPAR', 'LABNA', 'LABTNI', 'LABLIDOL', 'LABHCTX', 'LABLDH', 'LABURIC', 'LABLAC', 'LABSPLAC', 'LABFIB', 'LABUSPG']


In [8]:



'''
Each lab 
    -> all its orders in 2014/07-2017/06 (implicit) 
    -> {time since last order:cnts} (cached)
    -> {0-1 days: cnt, 1-3 days: ...} 
    -> barh
'''
lab2stats = {}
columns = ['< 24 hrs', '[24 hrs, 3 days)', '[3 days, 1 week)', '>= 1 week']

if os.path.exists(cached_result_path) and use_cached_fig_data:
    lab2stats_pd = pd.read_csv(cached_result_path)
    lab2stats_pd = lab2stats_pd.rename(columns={'< 1 day':'< 24 hrs',
                                                '1-3 days':'[24 hrs, 3 days)',
                                                '3-7 days':'[3 days, 1 week)',
                                                '> 7 days':'>= 1 week'})
    lab2stats = lab2stats_pd.set_index('lab').to_dict(orient='index')

else:
    for lab in labs:
        print 'Getting Order Intensities of lab %s..'%lab

        df_lab = stats_utils.get_queried_lab(lab, lab_type, time_limit=DEFAULT_TIMELIMIT)

        dict_lab = stats_utils.get_floored_day_to_number_orders_cnts(lab, df_lab)

        day_counts = [0] * 4
        tot_cnt = 0
        for time, cnt in dict_lab.items():
            if 0 <= time < 1:
                day_counts[0] += cnt
            elif 1 <= time < 3:
                day_counts[1] += cnt
            elif 3 <= time < 7:
                day_counts[2] += cnt
            elif time >= 7:
                day_counts[3] += cnt
            else:
                print "time is out of range:", time
            tot_cnt += cnt

        lab2stats[lab] = dict(zip(columns, day_counts))

    df_res = pd.DataFrame.from_dict(lab2stats, orient='index').reset_index().rename(columns={'index':'lab'})

    df_res.to_csv(cached_result_path, index=False)

for lab, stats in lab2stats.items():
    stats['total'] = float(sum(stats.values()))

if scale_method == 'by_scale':
    labs_ordered = sorted(labs, key=lambda x: lab2stats[x]['total'], reverse=True) # 
else:
    labs_ordered = sorted(labs, key=lambda x: lab2stats[x]['< 24 hrs']/lab2stats[x]['total'], reverse=True)
    
labs_toplots = [labs_ordered]

In [9]:
lab2stats_pd.shape

(20, 5)

In [10]:
def plot_order_intensities_barh(lab, time_since_last_order_binned, columns, scale,
                                labeling=True, lab_ind=None):

    alphas = [0.7, 1, 1, 0.7]
    colors = ['red', 'green', 'orange', 'blue']

    local_lab_descriptions = {
        'LABCBCD': 'CBC WITH\nDIFF',
        'LABPHOS': 'PHOSPHORUS\nSERUM/PLASMA',
        'LABA1C': 'HEMOGLOBIN\nA1C',
        'LABALB': 'ALBUMIN\nSERUM/PLASMA',
        'LABTSH': 'THYROID\nSIMULATING\nHORMONE',
        'LABESRP': 'SEDIMENTATION\nRATE (ESR)'
    }


    pre_sum = 0
    pre_sums = [pre_sum]
    
    for key in columns:
        pre_sum += time_since_last_order_binned[key]
        pre_sums.append(pre_sum)
    
    print pre_sums
        
#     print lab, time_since_last_order_binned.items()

    for i in [3,2,1,0]:
        key = columns[i]
        pre_sum = pre_sums[i+1]-pre_sums[i]
        left = pre_sums[i]

        lab_descriptions['LABTSH'] = 'TSH'
        lab_descriptions['LABESRP'] = 'Sedimentation Rate'
        lab_descriptions['LABCBCD'] = 'CBC w/ Diff'
        lab_descriptions['LABPTT'] = 'PTT'
        lab_descriptions['LABHEPAR'] = 'Heparin Activity Level'
        lab_descriptions['LABLIDOL'] = 'Lidocaine Level'
        lab_desciption = lab_descriptions.get(lab, lab)
        
        
        
#         print 'pre_sum, left:', pre_sum, left

        if labeling:
            plt.barh([lab_desciption], pre_sum, left=left, color=colors[i], alpha=alphas[i], label=key)
        else:
            plt.barh([lab_desciption], pre_sum, left=left, color=colors[i], alpha=alphas[i])

In [11]:


for ind_toplot, labs_toplot in enumerate(labs_toplots):
    fig, ax = plt.subplots(figsize=(8, .3 + 5.7/20.*len(labs))) #5+ 3./20.*len(labs)
    # 8, 3.435 for guideline
    # 8, 6 for freq labs
    # 6 - 3.435 = 2.565
    # Size for saturation is 7, 2.565 ?
    print .3 + 5.7/20.*len(labs)
    for i, lab in enumerate(labs_toplot[::-1]):

        time_since_last_order_binned = lab2stats[lab]


        for time, cnt in time_since_last_order_binned.items():
            if time == 'total':
                continue
            if scale_method == 'normalize':
                time_since_last_order_binned[time] = cnt/time_since_last_order_binned['total']

            elif scale_method == 'by_scale':
                time_since_last_order_binned[time] = cnt*scale

        if i == 0:
            plot_order_intensities_barh(lab, time_since_last_order_binned,
                                             columns=columns,
                                             labeling=True,
                                             lab_ind=i,
                                             scale=scale)
        else:
            plot_order_intensities_barh(lab, time_since_last_order_binned,
                                             columns=columns,
                                             labeling=False,
                                             lab_ind=i,
                                             scale=scale)

    if scale_method == 'normalize':
        ax.set_xticklabels(['{:,.0%}'.format(x) for x in np.linspace(0,1,num=6)])
        plt.tick_params('x', labelsize=13)
        plt.tick_params('y', labelsize=11)
        plt.xlim([0,1])
    else:
        plt.tick_params('x', labelsize=15) #12
        plt.tick_params('y', labelsize=13) #10

    if result_label != 'labs_guideline':
        plt.xlabel('Number of orders per 100 patient encounters', fontsize=14) #'Order number between 2014/07-2017/06'

    if include_legend:
        plt.legend(prop={'size': 12})
    else:
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("right")
        pass
    # plt.xscale('log')

    plt.tight_layout()
    cached_result_folderpath = cached_result_foldername + 'Order_Intensities_%s_%i_%s.png'%(lab_type,ind_toplot,result_label)

    if to_annotate_percentages:
        cached_result_folderpath.replace('.png', '_formal_1.png')
        # plt.xlim([0,5000])

    plt.savefig(cached_result_folderpath)
    plt.clf()

6.0
[0, 66.7519566526189, 67.74533413606261, 68.01625526791089, 76.06863335340157]
[0, 70.27393136664661, 102.13726670680315, 104.62071041541242, 147.62191450933176]
[0, 91.97772426249249, 99.05177603853102, 101.65562913907286, 152.45334136062615]
[0, 76.77603853100543, 93.82901866345576, 103.61228175797713, 187.1312462372065]
[0, 113.6062612883805, 191.87236604455148, 195.83082480433475, 229.0487658037327]
[0, 155.5689343768814, 210.09933774834437, 213.47080072245635, 288.4708007224564]
[0, 266.96267308850094, 274.0216736905479, 275.25586995785676, 314.90066225165566]
[0, 313.4708007224564, 325.7977122215533, 327.4683925346177, 360.9723058398555]
[0, 328.371462974112, 357.3148705599037, 363.74172185430467, 423.13365442504517]
[0, 135.83684527393137, 337.0860927152318, 347.9229379891632, 456.0656231186033]
[0, 302.81456953642385, 323.44972907886813, 334.7832630945214, 534.1210114388923]
[0, 607.1342564720048, 660.7615894039735, 671.5683323299218, 752.9650812763396]
[0, 793.151715833835

In [12]:
#lab2stats_pd.set_index('lab')

In [13]:
columns_reordered = ['< 24 hrs', '[24 hrs, 3 days)', '[3 days, 1 week)', '>= 1 week'][::-1]
lab2stats_pd.set_index('lab')[columns_reordered].plot.barh(width=0.9, figsize=(8,8))
plt.xlabel('number of orders')
plt.tight_layout()
plt.savefig(cached_result_folderpath.replace('.png', '_unstacked.png'))
plt.clf()

## Figure 2.2

## Normality Saturations

In [ ]:


max_repeat = 5
labs=['K', 'CR', 'NA', 'WBC', 'HGB']

if not os.path.exists(figure2_folderpath):
    os.mkdir(figure2_folderpath)

if os.path.exists(figure2_folderpath + 'lab2cnt.csv'):
    lab2cnt_pd = pd.read_csv(figure2_folderpath + 'lab2cnt.csv', keep_default_na=False)\
        .set_index('lab')
    lab2cnt_pd.columns = lab2cnt_pd.columns.astype(int)
    lab2cnt = lab2cnt_pd.to_dict(orient='index')

    lab2frac_pd = pd.read_csv(figure2_folderpath + 'lab2frac.csv', keep_default_na=False).set_index('lab')
    lab2frac_pd.columns = lab2frac_pd.columns.astype(int)
    lab2frac = lab2frac_pd.to_dict(orient='index')
else:

    lab2cnt, lab2frac = {}, {}
    import pickle
    cur_cnt_folderpath = 'Normality_Saturations_Cnts'
    if not os.path.exists(cur_cnt_folderpath):
        os.mkdir(cur_cnt_folderpath)

    def save_obj(obj, path):
        with open(path, 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    def load_obj(path):
        with open(path, 'rb') as f:
            return pickle.load(f)

    for lab in labs:
        print 'Getting Normality Saturations for %s..' % lab

        cur_dict_name = "cur_dict_%s.pkl"%lab
        cur_dict_path = os.path.join(cur_cnt_folderpath, cur_dict_name)
        if not os.path.exists(cur_dict_path):
            df_lab = stats_utils.get_queried_lab(lab, lab_type, time_limit=stats_utils.DEFAULT_TIMELIMIT)

            if lab_type=='panel':
                df_lab = df_lab[df_lab['order_status'] == 'Completed']

            cur_dict = stats_utils.get_prevweek_normal__dict(df_lab, lab_type)

            save_obj(cur_dict, cur_dict_path)
        else:
            cur_dict = load_obj(cur_dict_path)

        normal_fractions = {}
        record_counts = {}
        for x in range(0, max_repeat + 1):
            if x in cur_dict:
                record_count = len(cur_dict[x])
                normal_fraction = np.divide(sum(cur_dict[x]), float(record_count))
            else:
                record_count = 0
                normal_fraction = float('nan')

            record_counts[x] = record_count
            normal_fractions[x] = (normal_fraction)
        lab2cnt[lab] = record_counts
        lab2frac[lab] = normal_fractions

    df_cnts = pd.DataFrame.from_dict(lab2cnt, orient='index').reset_index().rename(columns={'index': 'lab'})
    df_fracs = pd.DataFrame.from_dict(lab2frac, orient='index').reset_index().rename(columns={'index': 'lab'})
    #
    # print df_cnts
    # print df_fracs
    # quit()

    df_cnts.to_csv(figure2_folderpath + 'lab2cnt.csv', index=False)
    df_fracs.to_csv(figure2_folderpath + 'lab2frac.csv', index=False)

In [ ]:
lab2cnt_pd

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 3.25)) #6, 4.5 # 7, 2.565 #6.5, 3.75

#, '<', '>'
marker_types = ('o', 'v', '^', '8', 's', 'P', '*', 'X', 'D', 'd')

for k, lab in enumerate(labs):  # :

    non_empty_inds = []
    for i in range(0,max_repeat+1):
        if lab2frac[lab][i]=='':
            break
        non_empty_inds.append(i)
    y_s = [float(lab2frac[lab][i]) for i in non_empty_inds]
    print 'lab, y_s', lab, y_s
    plt.plot(non_empty_inds, y_s, '-'+marker_types[k], label=labDescriptions[lab])
    # l2, = plt.scatter(non_empty_inds, y_s, marker=marker_types[k])
    # plt.plot(y_s[0], '-'+marker_types[k], color=l2.get_color(), markerfacecolor=l1.get_color(), label='My plots')

plt.xticks(range(0, max_repeat + 1))
plt.xlabel('Consecutive normal results in the past 7 days', fontsize=14)
plt.yticks([0,0.2,0.4,0.6,0.8,1], ['0%', '20%', '40%', '60%', '80%', '100%'])
plt.tick_params('x', labelsize=15)  # 12
plt.tick_params('y', labelsize=13)  # 10
plt.ylabel("Normal rate", fontsize=14)
plt.ylim([-0.05, 1.05])
plt.legend(fontsize=13)
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
plt.tight_layout()
plt.savefig(figure2_folderpath + 'Negative_Saturations_%s'%(lab_type))
plt.clf()